In [1]:
import sys
if sys.platform == "darwin":  # michael's config
    # because Michael is using jupyter lab
    %load_ext lab_black
else:  # zade's config
    # because Zade is using jupyter notebook
    %load_ext nb_black

## Tasks

I have put several more utility functions into the coverage_utils module, have a look at all of them to understand what they do.

Update: I added the CoverageCalculator, making things much easier...

Next tasks are:

* [ ] combine below code into a function with the name `calc_coverage(region_name)` that returns the pandas dataframe with the results (basically combining below cells into a function) but add a region_name column (same for all rows), in preparation for next task.
* [ ] run a loop over all available region names (see below for how to get that list).
    * [ ] store each returned dataframe in a bucket again
    * [ ] combine the bucket into a larger dataframe that has all results
    * [ ] save as CSV file using df.to_csv(...)
* [ ] Develep a plot function with the cells below, that conveniently reads the data out the above stored large dataframe and creates the plot, using the options as below.

# CoverageCalculator class

I wrote a class because it's much more efficient to keep the fans, blotches and metadata in memory and not read it in everytime. 
Keep that in mind for the future: When you need to read a data object often, it is more efficient to write a class that keeps that data in memory while you change a parameter (here: obsid) for different analyses.

This class reads in the general data files once and you can then use it in a loop by setting the obsid value as shown below.

In [2]:
from coverage_utils import CoverageCalculator, get_obsids_for_region
from my_io import get_tilecoords, get_metadata, get_region_names

In [3]:
obsid = "ESP_011341_0980"

In [4]:
cc = CoverageCalculator()

Reading fans and blotches in...
Done.
Reading metadata...
Done.
Note that obsid must be set to a string value before this works.


In [5]:
cc.obsid = "ESP_012079_0945"

In [6]:
cc.covered_fraction

0.115

There's also the union itself and it's area, in case you need it:

In [7]:
cc.union.area

25519111.051403664

And the Ls value for the current obsid:

In [8]:
cc.Ls

214.785

# Region of Interest names

I added also a function to give you the list of obsids for a region name.

In [9]:
obsids = get_obsids_for_region("ithaca")
obsids

124    ESP_011931_0945
125    ESP_012063_0945
126    ESP_020357_0950
127    ESP_011350_0945
128    ESP_020476_0950
129    ESP_021491_0950
130    ESP_020779_0950
131    ESP_012076_0945
132    ESP_012643_0945
133    ESP_020146_0950
134    ESP_012854_0945
135    ESP_011351_0945
136    ESP_011404_0945
137    ESP_011403_0945
138    ESP_020568_0950
139    ESP_012858_0855
140    PSP_003822_0945
141    PSP_002675_0945
142    PSP_003466_0945
143    PSP_004033_0945
144    PSP_003309_0945
145    PSP_003453_0945
146    PSP_003308_0945
147    PSP_003677_0945
148    PSP_003310_0855
149    PSP_003176_0945
150    PSP_003796_0950
151    PSP_002622_0945
152    PSP_003730_0945
153    PSP_003193_0850
154    PSP_003756_0945
155    PSP_003954_0945
156    PSP_004178_0945
157    PSP_004891_0945
158    PSP_004666_0945
159    ESP_029548_0945
160    ESP_038040_0950
161    ESP_038462_0950
162    ESP_037842_0950
163    ESP_039359_0950
164    ESP_040216_0950
165    ESP_040189_0950
Name: obsid, dtype: object

## Restrict to a martian year

by using the MY parameter, you can restrict the obsid list to a specific Martian Year:

In [10]:
get_obsids_for_region("ithaca", 29)

124    ESP_011931_0945
125    ESP_012063_0945
127    ESP_011350_0945
131    ESP_012076_0945
132    ESP_012643_0945
134    ESP_012854_0945
135    ESP_011351_0945
136    ESP_011404_0945
137    ESP_011403_0945
139    ESP_012858_0855
Name: obsid, dtype: object

In [38]:
get_region_names().roi_name.unique()

array(['Macclesfield', 'unknown', 'Starburst', 'Manhattan_Classic',
       'Wellington', 'Albany', 'Bilbao', 'Ithaca', 'Portsmouth', 'Pisaq',
       'Manhattan_Frontinella', 'BuenosAires', 'Inca_City_Ridges',
       'Inca_City', 'Giza', 'Potsdam', 'Troy', 'Oswego_Edge', 'Halifax',
       'Caterpillar', 'Rochester', 'Manhattan_Cracks', 'Schenectady',
       'Binghamton', 'Atka', 'Cortland', 'Geneseo', 'Manhattan2'],
      dtype=object)

In [105]:
bucket = []
roi = "ithaca"
for MY in [29, 30]:
    print(f"Martian year: {MY}")
    for obsid in get_obsids_for_region(roi, MY):
        print(f"Calculating obsid {obsid}")
        d = {}
        d["obsid"] = obsid
        d["MY"] = MY
        cc.obsid = obsid
        d["fraction"] = cc.covered_fraction
        d["Ls"] = cc.Ls
        bucket.append(d)

Martian year: 29
Calculating obsid ESP_011931_0945
Calculating obsid ESP_012063_0945
Calculating obsid ESP_011350_0945
Calculating obsid ESP_012076_0945
Calculating obsid ESP_012643_0945
Calculating obsid ESP_012854_0945
Calculating obsid ESP_011351_0945
Calculating obsid ESP_011404_0945
Calculating obsid ESP_011403_0945
Calculating obsid ESP_012858_0855
Martian year: 30
Calculating obsid ESP_020357_0950
Calculating obsid ESP_020476_0950
Calculating obsid ESP_021491_0950
Calculating obsid ESP_020779_0950
Calculating obsid ESP_020146_0950
Calculating obsid ESP_020568_0950


In [106]:
results = pd.DataFrame(bucket)

In [107]:
results

,Ls,MY,fraction,obsid
0,207.751,29,0.225,ESP_011931_0945
1,214.020,29,0.212,ESP_012063_0945
2,181.205,29,0.091,ESP_011350_0945
3,214.641,29,0.210,ESP_012076_0945
4,242.279,29,0.212,ESP_012643_0945
5,252.710,29,0.229,ESP_012854_0945
6,181.249,29,0.056,ESP_011351_0945
7,183.594,29,0.116,ESP_011404_0945
8,183.549,29,0.088,ESP_011403_0945
9,252.907,29,0.225,ESP_012858_0855


In [108]:
import hvplot
import hvplot.pandas
from bokeh.resources import INLINE

In [109]:
# fix the xticks for better plot comparison
xticks = list(range(170, 301, 10))
xticks

[170, 180, 190, 200, 210, 220, 230, 240, 250, 260, 270, 280, 290, 300]

In [110]:
savepath = Path(
    "/Users/klay6683/Dropbox/data/planet4/p4_analysis/coverage_plots"
)

In [111]:
plot = results.hvplot(
    x="Ls",
    y="fraction",
    by="MY",
    kind="scatter",
    title=roi,
    xlim=(170, 300),
    ylim=(-0.05, 0.5),
    grid=True,
    xticks=xticks,
    width=800,
    height=400,
)
# hvplot.show(plot)
for ext in ["png", "html"]:
    hvplot.save(plot, str(savepath / f"{roi}_coverage.{ext}"), resources=INLINE)

In [25]:
metadata = get_metadata()

In [26]:
metadata.columns

Index(['OBSERVATION_ID', 'IMAGE_CENTER_LATITUDE', 'IMAGE_CENTER_LONGITUDE',
       'SOLAR_LONGITUDE', 'START_TIME', 'map_scale', 'north_azimuth',
       '# of tiles'],
      dtype='object')